In [1]:
### Notebook modified from arthur girard - agirard@ethz.ch (https://gitlab.ethz.ch/arthurgirard/datadriveturbineperformanceanalysis/-/blob/main/Simple_ANN_CustomLoss_commented.py)

In [1]:
### Generate a dataframe for training from .csv file
import pandas as pd
import glob
import numpy as np
import seaborn as sns
color_palette = sns.color_palette()
sns.set_context('talk')
import matplotlib.pyplot as plt
import dswe
from numpy import absolute
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import zscore
from sklearn.cluster import DBSCAN

In [2]:
#reads a CSV file from the given file path and returns a Pandas DataFrame 
def generate_dataframe(filepath):
  dataframe = pd.read_csv(filepath)
  return dataframe

In [3]:
### Mapping month and day/night
def map_time(df):
    df['time'] = pd.to_datetime(df['time'], format='%m/%d/%Y %H:%M')
    # Create a new column 'month'
    df['month'] = df['time'].dt.month_name()
    #Create a new column 'day or night'
    #Considering that the day is between 6am and 6pm 
    df['Day.Night'] = df['time'].apply(lambda x: 'Day' if x.hour > 6 and x.hour < 18 else 'Night')
    time_mapping = {"Day": 1, "Night": 0}
    month_mapping =  {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                  'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
    df = df.replace(time_mapping)
    df = df.replace(month_mapping)
    df.drop("time", axis = 1, inplace = True)
    return df

In [4]:
def filter_split_df_input_output(df, list_input, list_output):
    
    db = DBSCAN(eps=0.2, min_samples=10).fit(df[["wind_speed", "power"]])
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    
    df = df[db.labels_ == 0]

    x = df[list_input]
    y = df[list_output]
    x.fillna(0)
    y.fillna(0)
    
    return (x, y)

In [5]:
def std_scale(df, df_reference):
    return((df-df_reference.mean())/df_reference.std())

In [6]:
def minmax_scale(df, df_reference):
    return((df-df_reference.min())/(df_reference.max()-df_reference.min()))

In [7]:
def calc_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true.values - y_pred) ** 2))

In [8]:
def print_performance(x_test, y_test, y_pred):
    plt.figure(figsize = [6,4])
    plt.scatter(x_test['wind_speed'], y_test,color = color_palette[0], label="Measurements",s=0.1)
    plt.scatter(x_test['wind_speed'], y_pred, color = color_palette[1], label="Model",s=0.1)
    plt.xlabel("Wind speed (m/s)")
    plt.ylabel("Output power (normalized)")
    plt.legend()
    sns.despine()
    plt.show()
    
    plt.figure(figsize = [6,4])
    plt.scatter(y_test, y_pred, color = color_palette[0],s=0.1)
    plt.xlabel("Measurements")
    plt.ylabel("Model")
    sns.despine()
    plt.show()
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = calc_rmse(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # print the results
    print("MSE:", mse)
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("R2 score:", r2)
    
    return()

In [39]:
def xgb_regressor(x, y, x_test):
    # define model
    model = xgb.XGBRegressor()
    model.fit(x, y)
    y_hat = model.predict(x_test)
    
    ## Uncomment the lines below to plot the feature importance:
    # fi = pd.DataFrame(data=model.feature_importances_,
    #          index=model.feature_names_in_,
    #          columns=['importance'])
    # plt.figure(figsize = [4, 3])
    # fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
    # plt.show()
    # sns.despine()

    return(y_hat)

In [20]:
def xgb_regressor_custom_loss(x, y, x_test, x_val, y_val):
    def custom_loss(y_pred, y_val):
        d = (y_val-y_pred)
        delta = 1  
        scale = 1 + (d / delta) ** 2
        scale_sqrt = np.sqrt(scale)
        grad = d / scale_sqrt + 2 * d
        hess = (1 / scale) / scale_sqrt + 2
        return grad, hess
    
    model = xgb.XGBRegressor(n_estimators=5000,
                       early_stopping_rounds=100,
                       objective = custom_loss,
                       max_depth=3,
                       eval_metric = 'rmse',
                       learning_rate=0.01)
    model.fit(x, y,
        eval_set=[(x, y), (x_val, y_val)], verbose = 500)
    y_hat = model.predict(x_test)
    return(y_hat)

In [14]:
'''
This is dbscan with xgb default and all parameters
'''


# modify the number below accordingly
#Participant ID number
id_number = 9
#Submission Number
Submission_number = 1

# load all the .csv file in the folder that end with "training"
# define the file paths of the CSV files 
path_training = r'../dataset/kelmarsh/training/*training.csv'
path2 = r'../dataset/penmanshiel/training/*training.csv'
files = glob.glob(path_training)+ glob.glob(path2)
# dataframe_training = pd.DataFrame()

column_list_input = ['wind_speed', 'wind_speed_sensor1', 'wind_speed_sensor1_SD', 'wind_speed_sensor2', 'wind_speed_sensor2_SD', 'density_adjusted_wind_speed', 'wind_direction', 'nacelle_position', 'wind_direction_SD', 'nacelle_position_SD', 'nacelle_ambient_temperature', 'TI','Day.Night']
column_list_ouput = ['power']

In [13]:
for file in files:
    dataframe_training = generate_dataframe(file)
    current_csv_file = file.split("/")[4]
    print(current_csv_file)
    
    dataframe_training = map_time(dataframe_training)    
    dataframe_training_scaled = std_scale(dataframe_training, dataframe_training)
    x_train, y_train = filter_split_df_input_output(dataframe_training_scaled, column_list_input, column_list_ouput)
    
    # load the real test data
    dataframe_test = pd.DataFrame()
    test_file_name = file.replace("training", "test")
    dataframe_test = generate_dataframe(test_file_name)
    dataframe_test = map_time(dataframe_test)  
    dataframe_test_scaled = std_scale(dataframe_test[column_list_input], dataframe_training[column_list_input])

    y_pred = xgb_regressor(x_train, y_train, dataframe_test_scaled)
    predicted_power_rescaled = (y_pred * dataframe_training[column_list_ouput].values.std()) + dataframe_training[column_list_ouput].values.mean()

    df_test = pd.read_csv(test_file_name)
    # create a numpy array of float32 of shape [None, 1]
    # fill the second column with the prediction array
    df_test.iloc[:, 1] = predicted_power_rescaled
    # save the DataFrame as a new csv file
    csv_name = test_file_name.split("/")[4]
    prefix = str(id_number) + '_#' + str(Submission_number) + '_'
    new_file_name = '/'.join(test_file_name.split("/")[0:4]) + "/" + prefix + csv_name
    # print(new_file_name)
    df_test.to_csv(new_file_name, index=False)


Kel_df2_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df6_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df4_training.csv


...

/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df7_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df6_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


In [17]:
'''
This is dbscan with xgb default and only wind parameters
'''


# modify the number below accordingly
#Participant ID number
id_number = 9
#Submission Number
Submission_number = 2

# load all the .csv file in the folder that end with "training"
# define the file paths of the CSV files 
path_training = r'../dataset/kelmarsh/training/*training.csv'
path2 = r'../dataset/penmanshiel/training/*training.csv'
files = glob.glob(path_training)+ glob.glob(path2)
# dataframe_training = pd.DataFrame()

column_list_input = ['wind_speed', 'wind_speed_sensor1', 'wind_speed_sensor1_SD', 'wind_speed_sensor2', 'wind_speed_sensor2_SD', 'density_adjusted_wind_speed']
column_list_ouput = ['power']

In [18]:
for file in files:
    dataframe_training = generate_dataframe(file)
    current_csv_file = file.split("/")[4]
    print(current_csv_file)
    
    dataframe_training = map_time(dataframe_training)    
    dataframe_training_scaled = std_scale(dataframe_training, dataframe_training)
    x_train, y_train = filter_split_df_input_output(dataframe_training_scaled, column_list_input, column_list_ouput)

    
    # load the real test data
    dataframe_test = pd.DataFrame()
    test_file_name = file.replace("training", "test")
    dataframe_test = generate_dataframe(test_file_name)
    dataframe_test = map_time(dataframe_test)  
    dataframe_test_scaled = std_scale(dataframe_test[column_list_input], dataframe_training[column_list_input])

    y_pred = xgb_regressor(x_train, y_train, dataframe_test_scaled)
    predicted_power_rescaled = (y_pred * dataframe_training[column_list_ouput].values.std()) + dataframe_training[column_list_ouput].values.mean()

    df_test = pd.read_csv(test_file_name)
    # create a numpy array of float32 of shape [None, 1]
    # fill the second column with the prediction array
    df_test.iloc[:, 1] = predicted_power_rescaled
    # save the DataFrame as a new csv file
    csv_name = test_file_name.split("/")[4]
    prefix = str(id_number) + '_#' + str(Submission_number) + '_'
    new_file_name = '/'.join(test_file_name.split("/")[0:4]) + "/" + prefix + csv_name
    # print(new_file_name)
    df_test.to_csv(new_file_name, index=False)


Kel_df2_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df6_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df4_training.csv


...

/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df7_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df6_training.csv


/tmp/ipykernel_29245/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()
/tmp/ipykernel_29245/1085349368.py:36: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


In [22]:
'''
This is dbscan with xgb custom loss and all parameters
'''

# modify the number below accordingly
#Participant ID number
id_number = 9
#Submission Number
Submission_number = 3

# load all the .csv file in the folder that end with "training"
# define the file paths of the CSV files 
path_training = r'../dataset/kelmarsh/training/*training.csv'
path2 = r'../dataset/penmanshiel/training/*training.csv'
files = glob.glob(path_training)+ glob.glob(path2)
# dataframe_training = pd.DataFrame()

column_list_input = ['wind_speed', 'wind_speed_sensor1', 'wind_speed_sensor1_SD', 'wind_speed_sensor2', 'wind_speed_sensor2_SD', 'density_adjusted_wind_speed', 'wind_direction', 'nacelle_position', 'wind_direction_SD', 'nacelle_position_SD', 'nacelle_ambient_temperature', 'TI','Day.Night']
column_list_ouput = ['power']

In [23]:
for file in files:
    dataframe_training = generate_dataframe(file)
    current_csv_file = file.split("/")[4]
    print(current_csv_file)
    
    dataframe_training = map_time(dataframe_training)    
    dataframe_training_scaled = std_scale(dataframe_training, dataframe_training)
    train_dataframe, val_dataframe = train_test_split(dataframe_training_scaled, test_size=0.2, random_state=12)
    x_train, y_train = filter_split_df_input_output(dataframe_training_scaled, column_list_input, column_list_ouput)
    x_val, y_val = filter_split_df_input_output(val_dataframe, column_list_input, column_list_ouput)
    
    # load the real test data
    dataframe_test = pd.DataFrame()
    test_file_name = file.replace("training", "test")
    dataframe_test = generate_dataframe(test_file_name)
    dataframe_test = map_time(dataframe_test)  
    dataframe_test_scaled = std_scale(dataframe_test[column_list_input], dataframe_training[column_list_input])

    y_pred = xgb_regressor_custom_loss(x_train, y_train, dataframe_test_scaled, x_val, y_val)
    predicted_power_rescaled = (y_pred * dataframe_training[column_list_ouput].values.std()) + dataframe_training[column_list_ouput].values.mean()

    df_test = pd.read_csv(test_file_name)
    # create a numpy array of float32 of shape [None, 1]
    # fill the second column with the prediction array
    df_test.iloc[:, 1] = predicted_power_rescaled
    # save the DataFrame as a new csv file
    csv_name = test_file_name.split("/")[4]
    prefix = str(id_number) + '_#' + str(Submission_number) + '_'
    new_file_name = '/'.join(test_file_name.split("/")[0:4]) + "/" + prefix + csv_name
    # print(new_file_name)
    df_test.to_csv(new_file_name, index=False)


Kel_df2_training.csv
[0]	validation_0-rmse:1.10538	validation_1-rmse:1.10806


/tmp/ipykernel_17353/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()


[500]	validation_0-rmse:0.07680	validation_1-rmse:0.07563
[1000]	validation_0-rmse:0.07267	validation_1-rmse:0.07162
[1500]	validation_0-rmse:0.07053	validation_1-rmse:0.06953
[2000]	validation_0-rmse:0.06904	validation_1-rmse:0.06807
[2500]	validation_0-rmse:0.06791	validation_1-rmse:0.06699
[3000]	validation_0-rmse:0.06699	validation_1-rmse:0.06614
[3500]	validation_0-rmse:0.06615	validation_1-rmse:0.06536
[4000]	validation_0-rmse:0.06562	validation_1-rmse:0.06490
[4500]	validation_0-rmse:0.06512	validation_1-rmse:0.06447
[4999]	validation_0-rmse:0.06468	validation_1-rmse:0.06408


/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df6_training.csv
[0]	validation_0-rmse:1.10518	validation_1-rmse:1.10150


...

/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df6_training.csv
[0]	validation_0-rmse:1.10561	validation_1-rmse:1.10318


/tmp/ipykernel_17353/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()


[500]	validation_0-rmse:0.06088	validation_1-rmse:0.06073
[1000]	validation_0-rmse:0.05661	validation_1-rmse:0.05656
[1500]	validation_0-rmse:0.05529	validation_1-rmse:0.05532
[2000]	validation_0-rmse:0.05426	validation_1-rmse:0.05447
[2500]	validation_0-rmse:0.05330	validation_1-rmse:0.05350
[3000]	validation_0-rmse:0.05255	validation_1-rmse:0.05276
[3500]	validation_0-rmse:0.05189	validation_1-rmse:0.05210
[4000]	validation_0-rmse:0.05131	validation_1-rmse:0.05145
[4500]	validation_0-rmse:0.05081	validation_1-rmse:0.05093
[4999]	validation_0-rmse:0.05036	validation_1-rmse:0.05047


/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


In [25]:
'''
This is dbscan with xgb custom loss and only wind parameters
'''

# modify the number below accordingly
#Participant ID number
id_number = 9
#Submission Number
Submission_number = 4

# load all the .csv file in the folder that end with "training"
# define the file paths of the CSV files 
path_training = r'../dataset/kelmarsh/training/*training.csv'
path2 = r'../dataset/penmanshiel/training/*training.csv'
files = glob.glob(path_training)+ glob.glob(path2)
# dataframe_training = pd.DataFrame()

column_list_input = ['wind_speed', 'wind_speed_sensor1', 'wind_speed_sensor1_SD', 'wind_speed_sensor2', 'wind_speed_sensor2_SD', 'density_adjusted_wind_speed']
column_list_ouput = ['power']

In [26]:
for file in files:
    dataframe_training = generate_dataframe(file)
    current_csv_file = file.split("/")[4]
    print(current_csv_file)
    
    dataframe_training = map_time(dataframe_training)    
    dataframe_training_scaled = std_scale(dataframe_training, dataframe_training)
    train_dataframe, val_dataframe = train_test_split(dataframe_training_scaled, test_size=0.2, random_state=12)
    x_train, y_train = filter_split_df_input_output(dataframe_training_scaled, column_list_input, column_list_ouput)
    x_val, y_val = filter_split_df_input_output(val_dataframe, column_list_input, column_list_ouput)

    # load the real test data
    dataframe_test = pd.DataFrame()
    test_file_name = file.replace("training", "test")
    dataframe_test = generate_dataframe(test_file_name)
    dataframe_test = map_time(dataframe_test)  
    dataframe_test_scaled = std_scale(dataframe_test[column_list_input], dataframe_training[column_list_input])

    y_pred = xgb_regressor_custom_loss(x_train, y_train, dataframe_test_scaled, x_val, y_val)
    predicted_power_rescaled = (y_pred * dataframe_training[column_list_ouput].values.std()) + dataframe_training[column_list_ouput].values.mean()

    df_test = pd.read_csv(test_file_name)
    # create a numpy array of float32 of shape [None, 1]
    # fill the second column with the prediction array
    df_test.iloc[:, 1] = predicted_power_rescaled
    # save the DataFrame as a new csv file
    csv_name = test_file_name.split("/")[4]
    prefix = str(id_number) + '_#' + str(Submission_number) + '_'
    new_file_name = '/'.join(test_file_name.split("/")[0:4]) + "/" + prefix + csv_name
    # print(new_file_name)
    df_test.to_csv(new_file_name, index=False)


Kel_df2_training.csv
[0]	validation_0-rmse:1.10538	validation_1-rmse:1.10806


/tmp/ipykernel_17353/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()


[500]	validation_0-rmse:0.08115	validation_1-rmse:0.07996
[1000]	validation_0-rmse:0.07943	validation_1-rmse:0.07828
[1500]	validation_0-rmse:0.07858	validation_1-rmse:0.07749
[2000]	validation_0-rmse:0.07769	validation_1-rmse:0.07667
[2500]	validation_0-rmse:0.07675	validation_1-rmse:0.07578
[3000]	validation_0-rmse:0.07596	validation_1-rmse:0.07506
[3500]	validation_0-rmse:0.07527	validation_1-rmse:0.07439
[4000]	validation_0-rmse:0.07463	validation_1-rmse:0.07373
[4500]	validation_0-rmse:0.07402	validation_1-rmse:0.07308
[4999]	validation_0-rmse:0.07352	validation_1-rmse:0.07260


/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Kel_df6_training.csv
[0]	validation_0-rmse:1.10518	validation_1-rmse:1.10150


...

/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled


Pen_df6_training.csv
[0]	validation_0-rmse:1.10561	validation_1-rmse:1.10318


/tmp/ipykernel_17353/898797509.py:5: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df['month'] = df['time'].dt.month_name()


[500]	validation_0-rmse:0.06752	validation_1-rmse:0.06619
[1000]	validation_0-rmse:0.06476	validation_1-rmse:0.06349
[1500]	validation_0-rmse:0.06423	validation_1-rmse:0.06288
[2000]	validation_0-rmse:0.06367	validation_1-rmse:0.06230
[2500]	validation_0-rmse:0.06312	validation_1-rmse:0.06175
[3000]	validation_0-rmse:0.06262	validation_1-rmse:0.06129
[3500]	validation_0-rmse:0.06205	validation_1-rmse:0.06078
[4000]	validation_0-rmse:0.06149	validation_1-rmse:0.06027
[4500]	validation_0-rmse:0.06093	validation_1-rmse:0.05980
[4999]	validation_0-rmse:0.06045	validation_1-rmse:0.05939


/tmp/ipykernel_17353/3725533942.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.iloc[:, 1] = predicted_power_rescaled
